In [1]:
"""
Purpose: To run the soma splitting 
on the processed neurons

"""


'\nPurpose: To run the soma splitting \non the processed neurons\n\n'

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

from importlib import reload

In [3]:
import datajoint_utils as du

INFO - 2021-01-13 23:25:57,486 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-13 23:25:57,487 - settings - Setting database.user to celiib
INFO - 2021-01-13 23:25:57,487 - settings - Setting database.password to newceliipass
INFO - 2021-01-13 23:25:57,494 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-13 23:25:57,495 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-13 23:25:57,508 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-13 23:25:57,810 - settings - Setting enable_python_native_blobs to True


In [4]:
test_mode = True

In [5]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-13 23:25:57,890 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-13 23:25:57,891 - settings - Setting database.user to celiib
INFO - 2021-01-13 23:25:57,891 - settings - Setting database.password to newceliipass
INFO - 2021-01-13 23:25:57,894 - settings - Setting enable_python_native_blobs to True


Sleeping 134 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-13 23:25:58,210 - settings - Setting enable_python_native_blobs to True


# Defining the Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 4363


In [8]:
#so that it will have the adapter defined
from datajoint_utils import DecompositionAdapter

In [9]:
DecompositionAdapter

datajoint_utils.DecompositionAdapter

In [10]:
(minnie.Decomposition() & dict(segment_id=864691135593563947)).delete()

AssertionError: Declaration context is not set

In [ ]:
key_source = minnie.Decomposition()
(key_source & dict(segment_id=864691135593563947)).delete()

In [ ]:
import numpy as np
import time
import proofreading_utils as pru

@schema
class NeuronSplitSuggestions(dj.Computed):
    definition="""
    -> minnie.Decomposition
    ---
    split_results: longblob #will store the results of how to split the limbs of neuron objects
    run_time=NULL : double                   # the amount of time to run (seconds)

    """

    key_source = minnie.Decomposition() & dict(segment_id=864691135593563947)
    
    def make(self,key):
        """
        Pseudocode for process:

        1) Fetch the neuron object
        2) Run the mutli-split suggestions
        3) Calculate the total time
        4) Write the results to the table
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        print(f"\n\n----- Working on Neuron {segment_id}-------")
        
        neuron_obj = (minnie.Decomposition & key).fetch1("decomposition")
        
        limb_results = pru.multi_soma_split_suggestions(neuron_obj,plot_intermediates=False)
        
        #7) Pass stats and file location to insert
        new_key = dict(key,
                       split_results = limb_results,
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for Splitting {segment_id} = {time.time() - whole_pass_time} ------")
    

# Running the Populate

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__neuron_split_suggestions'")
(curr_table)#.delete()# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

In [11]:
minnie.NeuronSplitSuggestions()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_results will store the results of how to split the limbs of neuron objects,run_time the amount of time to run (seconds)
864691134947393276,0,0.25,29.00,0,0,=BLOB=,109.5336
864691135012471798,0,0.25,29.00,0,0,=BLOB=,159.6045
864691135012571126,0,0.25,29.00,0,0,=BLOB=,270.8457
864691135012905110,0,0.25,29.00,0,0,=BLOB=,204.9058
864691135065018692,0,0.25,29.00,0,0,=BLOB=,196.6869
864691135065024068,0,0.25,29.00,0,0,=BLOB=,319.9624
864691135065032772,0,0.25,29.00,0,0,=BLOB=,110.7721
864691135065033540,0,0.25,29.00,0,0,=BLOB=,202.1275
864691135081745143,0,0.25,29.00,0,0,=BLOB=,118.0127
864691135155574372,0,0.25,29.00,0,0,=BLOB=,254.9494


In [ ]:
start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    NeuronSplitSuggestions.populate(reserve_jobs=True, suppress_errors=True)
else:
    NeuronSplitSuggestions.populate(reserve_jobs=True, suppress_errors=True)
print('Populate Done')

print(f"Total time for NeuronSplitSuggestions populate = {time.time() - start_time}")

In [ ]:
# import proofreading_utils as pru
# pru = reload(pru)
# du.create_suggested_splits_neuroglancer_spreadsheet(output_type="local")